In [1]:
"""
Esmé Middaugh
2019APRIL05

This notebook merges the yearly CDC prescription data into one csv, combining the separate files for 2006 through 2016
into one and appending a column with the appropriate year. 

This is the first part of my final project for GEOG 540. I wrote the original script for this slightly early in 
the semester for part of my research, but went back through and updated to make the code (and resulting data) cleaner. 
"""


'\nEsmé Middaugh\n2019APRIL05\n\nThis notebook merges the yearly CDC prescription data into one csv, combining the separate files for 2006 through 2016\ninto one and appending a column with the appropriate year. \n'

In [41]:
import os
import pandas as pd 
import statistics

In [54]:
## Reading in Prescription and Mortality Files
## Assumes you are in 'code' and 'data' is another folder in parent directory
pres_fname = "../raw_data/cdc_opioid_prescribing_rate.csv" 
with open (pres_fname) as f:
    prescription_df = pd.read_csv(f, na_values="–")
#pres_df.info()

mort_fname =  "../raw_data/NCHS_-_Drug_Poisoning_Mortality_by_County__United_States.csv"
with open (mort_fname) as f:
    mortality_df = pd.read_csv(f)
#mort_df.info()

In [63]:
#Cleaning the Prescription Data -- simple  
prescription_df = prescription_df.dropna()
clean_prescription_df = prescription_df[prescription_df['Year'] >= 2005]
clean_prescription_df.head()

,County,State,FIPS County Code,Prescribing Rate,Year
2,"Anchorage, AK",AK,2020,71.5,2006
7,"Fairbanks North Star, AK",AK,2090,54.7,2006
10,"Juneau, AK",AK,2110,95.3,2006
11,"Kenai Peninsula, AK",AK,2122,89.1,2006
12,"Ketchikan Gateway, AK",AK,2130,144.4,2006


In [57]:
def find_avg_mort(mort_range): #Calculating the average mortality rate given the Estimated Age-adjusted Death Rate 
    """
    Parameters:
    mort-range - a estimated range of average mortality rates in a county, given in the form of , '<2', #-#' or '30+'
    
    Returns:
    The mean of the mort-range, or 30 if the range is >30 
    """
    if '+' in mort_range:
        return 30.0
    elif '<' in mort_range:
        return float(statistics.mean([0,2]))
    else:
        separated = mort_range.split('-')
        lo, hi  = float(separated[0]), float(separated[1])
        return statistics.mean([lo, hi])
    
def clean_mortality_data(df):
    df['avg_mortality_rate'] = df['Estimated Age-adjusted Death Rate, 16 Categories (in ranges)'].apply(find_avg_mort)
    #For my research we only need the years 2006 to 2016 
    clean_df = df[df['Year'] >=2005]
    return clean_df 

In [59]:
clean_mortality_df = clean_mortality_data(mortality_df)
clean_mortality_df.head()
clean_mortality_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37632 entries, 6 to 56447
Data columns (total 8 columns):
FIPS                                                            37632 non-null int64
Year                                                            37632 non-null int64
State                                                           37632 non-null object
FIPS State                                                      37632 non-null int64
County                                                          37632 non-null object
Population                                                      37632 non-null object
Estimated Age-adjusted Death Rate, 16 Categories (in ranges)    37632 non-null object
avg_mortality_rate                                              37632 non-null float64
dtypes: float64(1), int64(3), object(4)
memory usage: 2.6+ MB


In [61]:
def merge_datasets(clean_prescription_df, clean_mortality_df): 
    ## Creating Merged Data Frame 
    df = pd.merge(clean_prescription_df, clean_mortality_df, left_on = ['FIPS County Code', 'Year'], right_on = ['FIPS', 'Year']) #Merging based on County Code and Year 
    df = df.drop(['FIPS County Code', 'County_y'], axis=1) #Dropping Duplicates

    #Renaming 
    df.columns = ['county', 'state_abbrv', 'prescription_rate', 'year', 'fips', 'state', 'fips_state', 'population', 'age_adjusted_mortality_range', 'avg_mortality_rate']

    #Reordering the columns
    df = df[['county', 'fips', 'state_abbrv', 'state', 'fips_state', 'year', 'population', 'prescription_rate', 'age_adjusted_mortality_range', 'avg_mortality_rate']]

    return df 

In [64]:
df = merge_datasets(clean_prescription_df, clean_mortality_df)
df.head()

,county,fips,state_abbrv,state,fips_state,year,population,prescription_rate,age_adjusted_mortality_range,avg_mortality_rate
0,"Anchorage, AK",2020,AK,Alaska,2,2006,"280,085",71.5,12-13.9,12.95
1,"Fairbanks North Star, AK",2090,AK,Alaska,2,2006,"90,545",54.7,8-9.9,8.95
2,"Juneau, AK",2110,AK,Alaska,2,2006,"30,808",95.3,8-9.9,8.95
3,"Kenai Peninsula, AK",2122,AK,Alaska,2,2006,"52,253",89.1,12-13.9,12.95
4,"Ketchikan Gateway, AK",2130,AK,Alaska,2,2006,"13,492",144.4,8-9.9,8.95


In [65]:
#Checking to make sure everything merged correctly 
df[df.fips == 1001]

,county,fips,state_abbrv,state,fips_state,year,population,prescription_rate,age_adjusted_mortality_range,avg_mortality_rate
8,"Autauga, AL",1001,AL,Alabama,1,2006,"51,328",134.8,6-7.9,6.95
2757,"Autauga, AL",1001,AL,Alabama,1,2007,"52,405",135.8,6-7.9,6.95
5497,"Autauga, AL",1001,AL,Alabama,1,2008,"53,277",144.9,6-7.9,6.95
8249,"Autauga, AL",1001,AL,Alabama,1,2009,"54,135",147.5,6-7.9,6.95
10993,"Autauga, AL",1001,AL,Alabama,1,2010,"54,660",151.7,8-9.9,8.95
13728,"Autauga, AL",1001,AL,Alabama,1,2011,"55,253",144.1,8-9.9,8.95
16468,"Autauga, AL",1001,AL,Alabama,1,2012,"55,175",157.8,8-9.9,8.95
19198,"Autauga, AL",1001,AL,Alabama,1,2013,"55,038",166.7,8-9.9,8.95
21946,"Autauga, AL",1001,AL,Alabama,1,2014,"55,290",145.3,10-11.9,10.95
24899,"Autauga, AL",1001,AL,Alabama,1,2015,"55,347",129.9,10-11.9,10.95


In [66]:
def yearly_change(source_df, source_col, change_col, sort_col, year_col):
    """ Calculates the change from the prior year for the given column
    Parameters:
    df - the pd.DataFrame you want to modify
    source_col - the column you want to calculate annual change for
    change_col - the column to hold these calculations
    
    Return:
    df - the dataframe with the updated columns 
    """
    #Sorting columns first so later iterating will work
    df = source_df.sort_values([sort_col,year_col],ascending=[True,True])

    #Getting the index for source column for use in iloc later
    source_col_index = df.columns.get_loc(source_col) 
    fips_index = df.columns.get_loc(sort_col)
    year_index = df.columns.get_loc(year_col)
    df[change_col] = None
    change_col_index = df.columns.get_loc(change_col)
    
    for row in range(len(df)):
        if df.iloc[row,fips_index] == df.iloc[row-1, fips_index] and df.iloc[row - 1, year_index] < df.iloc[row, year_index] : #checking that it is the same county code 
            df.iloc[row, change_col_index] = df.iloc[row, source_col_index] - df.iloc[row - 1, source_col_index]
    return df 

In [71]:
##Last bit of cleaning

#Actually calculating the changes per year 
df = yearly_change(df, 'avg_mortality_rate', 'change_mortality_rate', 'fips', 'year') 
df = yearly_change(df, 'prescription_rate', 'change_prescription_rate', 'fips', 'year')

## Fix the Population Data -- currently a string with a comma in it, need it to be an actual number
df['population'] = df['population'].map(lambda x: int(x.replace(',', '')))
df['population'][0]

#I would fix the fips code here but I discovered that writing it to a csv causes the problem to revert so just leaving
#as is and then will pre-process when I need it later. 

280085

Checking our final data frame to see if it looks right: 

In [74]:
print(df.head(5))
print(df.info())

            county  fips state_abbrv    state  fips_state  year  population  \
8      Autauga, AL  1001          AL  Alabama           1  2006       51328   
2757   Autauga, AL  1001          AL  Alabama           1  2007       52405   
5497   Autauga, AL  1001          AL  Alabama           1  2008       53277   
8249   Autauga, AL  1001          AL  Alabama           1  2009       54135   
10993  Autauga, AL  1001          AL  Alabama           1  2010       54660   

       prescription_rate age_adjusted_mortality_range  avg_mortality_rate  \
8                  134.8                        6-7.9                6.95   
2757               135.8                        6-7.9                6.95   
5497               144.9                        6-7.9                6.95   
8249               147.5                        6-7.9                6.95   
10993              151.7                        8-9.9                8.95   

      change_mortality_rate change_prescription_rate  
8      

In [76]:
#Finally, for this project we will just be looking at indiana 
indiana_df = df[df['state']=='Indiana']

In [77]:
#writing to CSV files so we can then use the clean data in our k-means clustering analysis and leaflet maps 
df.to_csv('../clean_data/opioid_data_2006-2016.csv')
indiana_df.to_csv('../clean_data/opioid_data_indiana_2006-2016.csv')